In [1]:
def extract_svo(nlp,sentence):
    doc = nlp(sentence)
    print("senmtence:",sentence)
    for token in doc:
        verb_subject =[]
        verb_aux_neg =[]
        dobj_tok = []
        
        if(token.dep_ in ('dobj','attr') and token.pos_ == 'NOUN' and token.head.pos_ in ('VERB')):
            verb_subject =  [tok for tok in token.head.lefts if token.dep_ in ('nsubj',)]
            verb_aux_neg = [tok for tok in token.head.lefts if token.dep_ in ("aux","neg",)]
            dobj_tok = [token]  
            return_text = ' '.join([token.text for token in verb_subject]) +' '.join([token.text for token in verb_aux_neg]) + "-v-" + token.head.text + "-o- " + ' '.join([token.text for token in dobj_tok])
            print("return_text:",return_text) 
            
    return


def refine_rule_results_aggregate(nlp,pd,df,column_list,input_file_name):
    my_columns = ['speaker']
    my_columns.extend(column_list)
    df = df[my_columns]
    print("my_df:",df.columns)
    for index,row in df.iterrows():
        for items in column_list:
            sent = row[items]
            sent_list=[]
            for sentence in sent.split("."):
                if(lookup_for_tech_terms(nlp,sentence)):
                    extract_svo(nlp,sentence)
                    sent_list.append(sentence)
            if(sent_list):
                df.at[index,items] = '.'.join(sent_list)
            else:
                df.at[index,items] = ""
                              
    output_file_path = input_file_name[0:input_file_name.rfind('.')]+"_EXTRACT.xlsx"
    try:
        with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
            
            df.to_excel(writer, sheet_name='one')
            print("file Saved")
   
        
    except Exception as e:
        print("Error while storing results:",e)
        writer.close()
    print("refine_rule_results : process completed")
    return

In [2]:
def lookup_for_tech_terms(nlp,my_conversation):
    doc = nlp(my_conversation)
    for ent in doc.ents:
        # print(my_conversation)
        return True
    return False

def refine_rule_results(nlp,pd,df,column_list,input_file_name):
    for index,row in df.iterrows():
        for items in column_list:
            sent = row[items]
            sent_list=[]
            for sentence in sent.split("."):
                if(lookup_for_tech_terms(nlp,sentence)):
                    sent_list.append(sentence)
            if(sent_list):
                df.at[index,items] = '.'.join(sent_list)
            else:
                df.at[index,items] = ""
                              
    output_file_path = input_file_name[0:input_file_name.rfind('.')]+"_EXTRACT.xlsx"
    try:
        with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
            
            df.to_excel(writer, sheet_name='one')
            print("file Saved")
   
        
    except Exception as e:
        print("Error while storing results:",e)
        writer.close()
    print("refine_rule_results : process completed")
    return

In [3]:
def read_processed_result_set(pd,input_file_name):
    processed_result_set = pd.read_excel(input_file_name)
    return processed_result_set

def read_file(nlp,pd,input_file_name):
    # input_file_name = "/home/axe/NLP/JSON-TRANSCRIPTS/GMT20210427-170250_Recording.transcript-US-87.xlsx"
    df = read_processed_result_set(pd,input_file_name)
    df.fillna("", inplace=True)
    column_list = ['subj_verb_dobj', 'subj_verb_pobj', 'subj_verb_dobj_pobj',
       'verb_tense', 'graph_brain', 'verb_tense_graph_brain',
       'graph_brain_verb_tense', 'graph_brain_subj_verb_dobj',
       'verb_tense_graph_brain_dobj', 'graph_brain_verb_tense_dobj']    
    # calling process function
    # refine_rule_results(nlp,pd,df,column_list,input_file_name)
    refine_rule_results_aggregate(nlp,pd,df,column_list,input_file_name)
    return


In [4]:
# import warnings.filterwarnings(action='once')

def main():
    from spacy import load
    import pandas as pd
    from os import listdir
    from os.path import isfile, join

    nlp = load('en_core_web_sm')
    output_pattern_JSONL_file = "/home/axe/NLP/terminilogy.jsonl"
    if("entity_ruler" not in nlp.pipe_names):
        ruler=nlp.add_pipe("entity_ruler").from_disk(output_pattern_JSONL_file)
    dir_path="/home/axe/NLP/JSON-TRANSCRIPTS/"
    vtt_file_list = [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f)) and f.endswith('.vtt')]
    for each_file in vtt_file_list:
        input_file_name = each_file[0:each_file.rfind('.')]+".xlsx"
        if(isfile(input_file_name)):
            print(each_file)
            read_file(nlp,pd,input_file_name)
        
        
    input_file_name = "/home/axe/NLP/JSON-TRANSCRIPTS/GMT20210503-185720_Recording.transcript-SI-8.xlsx"
    # read_file(nlp,pd,input_file_name)
       
    print("Processes started")
    
    
if __name__ == "__main__":
    main()

/home/axe/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-16 13:12:50.684623: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 13:12:50.684661: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/axe/.local/lib/python3.8/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details

/home/axe/NLP/JSON-TRANSCRIPTS/GMT20210816-210652_Recording.transcript-TWM-37.vtt
my_df: Index(['speaker', 'subj_verb_dobj', 'subj_verb_pobj', 'subj_verb_dobj_pobj',
       'verb_tense', 'graph_brain', 'verb_tense_graph_brain',
       'graph_brain_verb_tense', 'graph_brain_subj_verb_dobj',
       'verb_tense_graph_brain_dobj', 'graph_brain_verb_tense_dobj'],
      dtype='object')
senmtence: you can do people
return_text: -v-do-o- people
senmtence: you can do people
return_text: -v-do-o- people
senmtence: can do people
return_text: -v-do-o- people
senmtence: you can do people now
return_text: -v-do-o- people
senmtence: can do people
return_text: -v-do-o- people
senmtence: you can do people
return_text: -v-do-o- people
senmtence: can do people
return_text: -v-do-o- people
senmtence: i'm going to hold up what i call a family tree
senmtence: 
ca n't know the name of every kid i ca n't recall that the age of every kid , especially on the spot
return_text: -v-know-o- name
return_text: -v-rec